In [ ]:
from pathlib import Path
import pandas as pd
idx_slice = pd.IndexSlice
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Comment out for debugging and development
warnings.simplefilter(action='ignore', category=DeprecationWarning) # Comment out for debugging and development
import plotly.graph_objects as go

import sys
sys.path.append('..')
from scripts.plot_helpers import (
    chdir_to_parent_dir,
    read_stats_dict,
    prepare_dataframe,
    nice_title,
    save_plotly_fig)

chdir_to_parent_dir()

In [ ]:
run_name_prefix = "H2G_A" # Experiment name

sdir = Path.cwd() / "results"/ f"{run_name_prefix}_summary_20250523"
sdir.mkdir(exist_ok=True, parents=True)



## Marginal prices | H2 export price

In [ ]:
print(mean_marginal_prices["H2 export bus"])

print(mean_marginal_prices["H2 export bus"].index.names)

idx_level = mean_marginal_prices["H2 export bus"].index.levels
for i, level in enumerate(idx_level):
    print(f"Level {i} ({mean_marginal_prices['H2 export bus'].index.names[i]}): {list(level)}")

print(mean_marginal_prices["H2 export bus"].index.get_level_values("year").unique())



In [ ]:
# df = prepare_dataframe(
#     mean_marginal_prices["H2 export bus"], 
#     idx_slice[["H2G_A"],:,[2035],:]
# )

idx = pd.IndexSlice

df = prepare_dataframe(
    mean_marginal_prices["H2 export bus"],
    idx[["H2G_A"], :,[2035,2050],:]
)


df = df.query("variable in ['H2 export bus']")


In [ ]:
def marginal_price_dumbbell_fig(df, year, data_start="0.1MtH2export", data_end="0.7MtH2export"):

    df = df.copy()
    df = df[df.year==year]
    countries = df["country"].unique()#


    data = {"line_x": [], "line_y": [], "data_start": [], "data_end": [], "colors": [], "years": [], "countries": []}

    for country in countries:
        data["data_start"].extend([df.loc[(df.scen == data_start) & (df.country == country)]["value"].values[0]])
        data["data_end"].extend([df.loc[(df.scen == data_end) & (df.country == country)]["value"].values[0]])
        data["line_x"].extend(
            [
                df.loc[(df.scen == data_start) & (df.country == country)]["value"].values[0],
                df.loc[(df.scen == data_end) & (df.country == country)]["value"].values[0],
                None,
            ]
        )

        data["line_y"].extend([country, country, None])


    fig = go.Figure(
        data=[
            go.Scatter(
                x=data["line_x"],
                y=data["line_y"],
                mode="lines",
                showlegend=False,
                marker=dict(
                    color="grey"
                )
            ),
            go.Scatter(
                x=data["data_start"],
                y=countries,
                mode="markers+text",
                name=data_start,
                text=data["data_start"],            
                textposition="top center",
                texttemplate="%{text:.1f}",
                textfont=dict(size=11),
                marker=dict(
                    color="#A6BCC9",
                    size=13
                )

            ),
            go.Scatter(
                x=data["data_end"],
                y=countries,
                mode="markers+text",
                name=data_end,
                text=data["data_end"],
                textposition="top center",
                texttemplate="%{text:.1f}",
                textfont=dict(size=11),
                marker=dict(
                    color="#179c7d",
                    size=13
                )
            ),
        ]
    )

    fig.update_layout(
        title=dict(text=nice_title(f"Marginal price for H2 at export port in {year}", 
                                   "Per country and H2 export volume in €/MWh_H2_LHV")),
        width=800,
        height=500,
        legend_itemclick=False
    )
    return fig



In [ ]:
fig = marginal_price_dumbbell_fig(df, year=2050, data_start="0.7MtH2export", data_end="4.0MtH2export")

fig.show()

In [ ]:
idx = pd.IndexSlice

df = prepare_dataframe(
    mean_marginal_prices["H2 export bus"],
    idx[["H2G_A"], :,[2035],:]
)


df = df.query("variable in ['H2 export bus']")

In [ ]:
fig = marginal_price_dumbbell_fig(df, year=2035, data_start="0.1MtH2export", data_end="0.7MtH2export")
df.to_csv("results/marginal_prices.csv")
fig.show()